In [1]:
import scipy.io
data = scipy.io.loadmat('D:/本科研究生资料/硕士研究生/课程/机器学习与数据科学/作业/MNIST784x50x10.mat')

In [2]:
print(data['X'][300])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   3   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0  60   0 253 253
   0 215   0 139  37   0   0 253 105 253 255   0   0   0   0 253  62 253
 208   0 251   0   0 253 239 252   0 253 253 252   0 235 252   0 164 253
   0 250   0   0  81   0 247 252   0   0 252   0   0 253   0   0 110   0
   0   0   0  71 253   0  75   0  31 212 224  47   0 178 169  28   0 163
   0   0   0 103 128 200   0   0   0   0   0 252   0 253   0 234 204   0
 183  43 206   0   0 253 254 253  36   0   0   0 253   0 205 252   0 202
 254 171   0   0   0   0 254   0 128  43 254  70 228   0 253 253  16   0
 253 243 254 193   0  31   0   0   0  12   0   0   0 237 254   0 114   0
   0   0   0  39   0   0 181   0   0   0  36  79 115   0  33   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  99   0
   0 159   0   0   0   0   0 191  20   0   0  41   

In [3]:
def parse(data):
    data_set = data['X']
    label = data['index'][0]
    data_num = len(label)
    dimensions = len(data['X'])
    return data_set,label,data_num,dimensions

data_set,label,data_num,dimensions = parse(data)
print(dimensions)

784


In [4]:
from random import sample
def split_train_test(data_num):
    l = range(data_num)
    train_set = sorted(sample(l,int(data_num/2)))
    test_set = sorted(set(l) - set(train_set))
    return train_set,test_set

train_set,test_set = split_train_test(data_num)
print(train_set)
print(test_set)

[0, 1, 2, 4, 5, 6, 12, 13, 14, 15, 16, 18, 19, 20, 21, 23, 25, 26, 27, 29, 30, 32, 35, 36, 37, 38, 39, 40, 44, 45, 47, 49, 50, 54, 55, 56, 57, 59, 61, 62, 63, 64, 66, 68, 70, 72, 73, 74, 75, 76, 77, 78, 80, 82, 87, 88, 91, 93, 95, 98, 99, 101, 104, 108, 110, 113, 114, 115, 118, 119, 126, 130, 132, 135, 138, 139, 141, 142, 145, 147, 150, 151, 152, 156, 158, 160, 162, 164, 170, 171, 172, 175, 176, 177, 179, 181, 182, 186, 188, 194, 195, 196, 197, 202, 204, 209, 210, 211, 213, 214, 215, 216, 217, 218, 223, 225, 226, 227, 228, 230, 232, 236, 238, 240, 241, 243, 246, 247, 249, 250, 255, 260, 261, 268, 269, 272, 275, 277, 278, 279, 280, 281, 282, 285, 287, 293, 294, 295, 298, 299, 306, 313, 318, 320, 321, 323, 326, 328, 332, 334, 337, 338, 339, 341, 342, 344, 345, 347, 348, 351, 353, 359, 360, 361, 364, 370, 371, 373, 377, 378, 379, 381, 382, 383, 385, 386, 388, 389, 391, 392, 393, 394, 395, 397, 399, 400, 402, 403, 408, 410, 412, 413, 414, 415, 417, 418, 419, 422, 425, 426, 427, 428, 429, 4

In [5]:
def Odistance(x,y):
    distance = 0
    dimensions = len(x)
    for dim in range(dimensions):
        distance += (int(x[dim]) - int(y[dim]))**2
    return distance**0.5

#print(Odistance([1,1,1,1],[2,2,2,2]))

In [6]:
from cvxopt import matrix, solvers

def judge(k_list):
    k_index_list = [x[0] for x in k_list]
    class_ = [0]*11
    for index in k_index_list:
        class_[label[index]] += 1
    return class_.index(max(class_))

def judge2(k_list):
    class_ = [0]*11
    d1 = k_list[0][1]
    dk = k_list[-1][1]
    for index_distance in k_list:
        index = index_distance[0]
        di = index_distance[1]
        if dk - d1 != 0:
            wi = (dk - di)/(dk - d1)
        else:
            wi = 1
        class_[label[index]] += wi
    return class_.index(max(class_))

def judge3(k_list,x):
    k_index_list = [x[0] for x in k_list]
    k = len(k_index_list)
    class_ = [0]*11
    P = []
    q = []
    G = []
    h = []
    A = []
    b = []
    for index1 in k_index_list:
        p_row = []
        for index2 in k_index_list:
            p_value = 0.0
            for dim in range(dimensions):
                p_value += 2*data_set[dim][index1]*data_set[dim][index2]
            p_row.append(p_value)
        P.append(p_row)
        
    for index in k_index_list:
        q_value = 0.0
        for dim in range(dimensions):
            q_value += -2*x[dim]*data_set[dim][index]
        q.append(q_value)
        
    for i in range(k):
        G_row = [0.0]*k
        G_row[i] = -1.0
        G.append(G_row)
        
    h = [0.0]*k
    A = [1.0]*k 
    b = [1.0]
    
    P = matrix(P)
    q = matrix(q)
    G = matrix(G)
    h = matrix(h)
    A = matrix(A,(1,k))#原型为cvxopt.matrix(array,dims)，等价于A = matrix([[1.0],[1.0]]）
    b = matrix([1.0])
    
    W = solvers.qp(P,q,G,h,A,b)['x']
    for i in range(k):
        class_[label[k_index_list[i]]] += W[i]
    return class_.index(max(class_))

In [7]:
train_set,test_set = split_train_test(data_num)

In [39]:
train_set,test_set = split_train_test(data_num)
k = 50
real_class_list = []
predict_class_list = []
test_num = len(test_set)
hit_num = 0
for index1 in test_set:
    real_class = label[index1]
    real_class_list.append(real_class)
    x = []
    for i in range(dimensions):
        x.append(data_set[i][index1])
    distance_index_list = []
    for index2 in train_set:
        y = []
        for i in range(dimensions):
            y.append(data_set[i][index2])
        distance = Odistance(x,y)
        distance_index_list.append((index2,distance))
    distance_index_list.sort(key = lambda x : x[1]) #从小到大
    k_list = distance_index_list[0:k]
    
    predict_class = judge3(k_list,x)
    predict_class_list.append(predict_class)
    if predict_class == real_class:
        hit_num += 1
        
print("准确率为：",hit_num)
print(real_class_list)
print(predict_class_list)

     pcost       dcost       gap    pres   dres
 0: -3.5215e+06 -3.5215e+06  1e+02  1e+01  3e-07
 1: -3.5215e+06 -3.5215e+06  4e+01  3e+00  1e-07
 2: -3.5215e+06 -3.5215e+06  4e+01  3e+00  1e-07
 3: -3.4382e+06 -3.5226e+06  2e+05  3e+00  1e-07
 4: -3.4265e+06 -3.5471e+06  3e+05  2e+00  6e-08
 5: -3.4239e+06 -3.5097e+06  2e+05  1e+00  4e-08
 6: -3.4473e+06 -3.4743e+06  1e+05  6e-01  2e-08
 7: -3.4331e+06 -3.4193e+06  1e+05  4e-01  1e-08
 8: -3.4073e+06 -3.3784e+06  9e+04  2e-01  6e-09
 9: -3.3821e+06 -3.3603e+06  5e+04  8e-02  3e-09
10: -3.3649e+06 -3.3468e+06  3e+04  4e-02  1e-09
11: -3.3455e+06 -3.3404e+06  8e+03  9e-03  3e-10
12: -3.3387e+06 -3.3393e+06  6e+02  6e-05  2e-12
13: -3.3391e+06 -3.3392e+06  1e+01  9e-07  3e-14
14: -3.3391e+06 -3.3391e+06  4e-01  1e-08  4e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.3590e+06 -3.3590e+06  1e+02  1e+01  3e-07
 1: -3.3590e+06 -3.3590e+06  3e+01  3e+00  1e-07
 2: -3.3590e+06 -3.3589e+06  3e+01  3e+00  9e-0

     pcost       dcost       gap    pres   dres
 0: -3.0582e+06 -3.0582e+06  9e+01  8e+00  4e-07
 1: -3.0582e+06 -3.0582e+06  1e+01  1e+00  5e-08
 2: -3.0582e+06 -3.0582e+06  1e+01  1e+00  5e-08
 3: -3.0581e+06 -3.0570e+06  1e+02  1e+00  5e-08
 4: -3.0520e+06 -3.0526e+06  1e+04  1e+00  5e-08
 5: -3.0513e+06 -3.0485e+06  1e+04  8e-01  4e-08
 6: -3.0503e+06 -3.0419e+06  1e+04  7e-01  3e-08
 7: -3.0446e+06 -3.0278e+06  2e+04  5e-01  2e-08
 8: -3.0428e+06 -3.0188e+06  2e+04  5e-01  2e-08
 9: -3.0258e+06 -2.9939e+06  3e+04  3e-01  1e-08
10: -3.0185e+06 -2.9808e+06  3e+04  2e-01  1e-08
11: -2.9983e+06 -2.9632e+06  3e+04  1e-01  7e-09
12: -2.9859e+06 -2.9511e+06  3e+04  1e-01  5e-09
13: -2.9657e+06 -2.9390e+06  3e+04  6e-02  3e-09
14: -2.9560e+06 -2.9349e+06  2e+04  4e-02  2e-09
15: -2.9424e+06 -2.9301e+06  2e+04  2e-02  1e-09
16: -2.9270e+06 -2.9277e+06  2e+03  1e-03  4e-11
17: -2.9273e+06 -2.9274e+06  7e+01  3e-05  1e-12
18: -2.9274e+06 -2.9274e+06  4e+00  5e-07  2e-14
19: -2.9274e+06 -2.92

     pcost       dcost       gap    pres   dres
 0: -3.8184e+06 -3.8184e+06  9e+01  9e+00  3e-07
 1: -3.8184e+06 -3.8184e+06  2e+01  2e+00  7e-08
 2: -3.8184e+06 -3.8184e+06  2e+01  2e+00  6e-08
 3: -3.8054e+06 -3.8017e+06  3e+04  2e+00  6e-08
 4: -3.7848e+06 -3.7776e+06  7e+04  2e+00  6e-08
 5: -3.7810e+06 -3.7675e+06  7e+04  1e+00  5e-08
 6: -3.7731e+06 -3.7426e+06  7e+04  1e+00  4e-08
 7: -3.7575e+06 -3.7133e+06  6e+04  7e-01  2e-08
 8: -3.7234e+06 -3.6628e+06  7e+04  4e-01  2e-08
 9: -3.6991e+06 -3.6111e+06  8e+04  3e-01  1e-08
10: -3.6549e+06 -3.5626e+06  8e+04  2e-01  7e-09
11: -3.5909e+06 -3.5317e+06  5e+04  9e-02  3e-09
12: -3.5556e+06 -3.5196e+06  4e+04  5e-02  2e-09
13: -3.5251e+06 -3.5144e+06  1e+04  1e-02  5e-10
14: -3.5176e+06 -3.5135e+06  6e+03  5e-03  2e-10
15: -3.5123e+06 -3.5130e+06  7e+02  2e-16  1e-16
16: -3.5129e+06 -3.5129e+06  1e+01  2e-16  1e-16
17: -3.5129e+06 -3.5129e+06  1e-01  3e-16  1e-16
Optimal solution found.
     pcost       dcost       gap    pres   dre

     pcost       dcost       gap    pres   dres
 0: -3.2337e+06 -3.2337e+06  8e+01  9e+00  3e-07
 1: -3.2337e+06 -3.2337e+06  1e+01  2e+00  6e-08
 2: -3.2337e+06 -3.2337e+06  1e+01  2e+00  6e-08
 3: -3.2275e+06 -3.2254e+06  1e+04  2e+00  6e-08
 4: -3.2122e+06 -3.2029e+06  4e+04  2e+00  6e-08
 5: -3.2114e+06 -3.1897e+06  5e+04  2e+00  6e-08
 6: -3.2029e+06 -3.1658e+06  7e+04  1e+00  5e-08
 7: -3.1949e+06 -3.1454e+06  7e+04  1e+00  5e-08
 8: -3.1823e+06 -3.1116e+06  7e+04  1e+00  4e-08
 9: -3.1484e+06 -3.0447e+06  8e+04  7e-01  2e-08
10: -3.0667e+06 -2.9578e+06  1e+05  4e-01  1e-08
11: -2.9919e+06 -2.9086e+06  1e+05  2e-01  8e-09
12: -2.9511e+06 -2.8924e+06  8e+04  1e-01  5e-09
13: -2.9134e+06 -2.8841e+06  5e+04  6e-02  2e-09
14: -2.8701e+06 -2.8798e+06  1e+04  1e-03  3e-11
15: -2.8763e+06 -2.8768e+06  5e+02  2e-05  8e-13
16: -2.8767e+06 -2.8767e+06  9e+00  2e-07  8e-15
17: -2.8767e+06 -2.8767e+06  6e-01  3e-09  2e-16
Optimal solution found.
     pcost       dcost       gap    pres   dre

     pcost       dcost       gap    pres   dres
 0: -5.5878e+06 -5.5878e+06  1e+02  1e+01  2e-07
 1: -5.5878e+06 -5.5878e+06  4e+01  3e+00  7e-08
 2: -5.5878e+06 -5.5877e+06  4e+01  3e+00  7e-08
 3: -5.4097e+06 -5.6062e+06  4e+05  3e+00  7e-08
 4: -5.4733e+06 -5.5634e+06  4e+05  2e+00  5e-08
 5: -5.4668e+06 -5.4937e+06  4e+05  2e+00  4e-08
 6: -5.4626e+06 -5.4118e+06  4e+05  1e+00  3e-08
 7: -5.4247e+06 -5.3153e+06  2e+05  7e-01  1e-08
 8: -5.3659e+06 -5.2204e+06  2e+05  4e-01  9e-09
 9: -5.2625e+06 -5.1363e+06  2e+05  2e-01  5e-09
10: -5.1673e+06 -5.0989e+06  9e+04  9e-02  2e-09
11: -5.1148e+06 -5.0868e+06  6e+04  4e-02  9e-10
12: -5.0784e+06 -5.0804e+06  6e+03  2e-03  4e-11
13: -5.0792e+06 -5.0793e+06  1e+02  2e-05  5e-13
14: -5.0793e+06 -5.0793e+06  3e+00  3e-07  6e-15
15: -5.0793e+06 -5.0793e+06  7e-02  3e-09  1e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -6.7324e+06 -6.7324e+06  9e+01  8e+00  2e-07
 1: -6.7324e+06 -6.7324e+06  1e+01  1e+00  3e-0

...

     pcost       dcost       gap    pres   dres
 0: -4.8498e+06 -4.8498e+06  9e+01  8e+00  3e-07
 1: -4.8498e+06 -4.8498e+06  1e+01  1e+00  4e-08
 2: -4.8498e+06 -4.8498e+06  1e+01  1e+00  4e-08
 3: -4.8427e+06 -4.8346e+06  1e+04  1e+00  4e-08
 4: -4.8383e+06 -4.8144e+06  3e+04  1e+00  4e-08
 5: -4.8222e+06 -4.7901e+06  6e+04  1e+00  4e-08
 6: -4.7408e+06 -4.7831e+06  3e+05  1e+00  3e-08
 7: -4.6830e+06 -4.7663e+06  5e+05  9e-01  3e-08
 8: -4.6883e+06 -4.6782e+06  5e+05  7e-01  2e-08
 9: -4.6733e+06 -4.5870e+06  3e+05  4e-01  1e-08
10: -4.6269e+06 -4.5088e+06  2e+05  2e-01  8e-09
11: -4.5718e+06 -4.4388e+06  2e+05  1e-01  5e-09
12: -4.4442e+06 -4.3822e+06  1e+05  5e-02  2e-09
13: -4.3601e+06 -4.3716e+06  1e+04  2e-04  6e-12
14: -4.3686e+06 -4.3690e+06  4e+02  5e-06  2e-13
15: -4.3689e+06 -4.3689e+06  7e+00  6e-08  2e-15
16: -4.3689e+06 -4.3689e+06  7e-02  6e-10  1e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.6339e+06 -3.6339e+06  1e+02  9e+00  3e-0

     pcost       dcost       gap    pres   dres
 0: -4.5458e+06 -4.5458e+06  8e+01  8e+00  3e-07
 1: -4.5458e+06 -4.5458e+06  9e+00  9e-01  3e-08
 2: -4.5458e+06 -4.5458e+06  9e+00  8e-01  3e-08
 3: -4.5453e+06 -4.5434e+06  1e+03  8e-01  3e-08
 4: -4.5129e+06 -4.5516e+06  7e+04  8e-01  3e-08
 5: -4.5158e+06 -4.5450e+06  7e+04  7e-01  2e-08
 6: -4.5163e+06 -4.5198e+06  7e+04  6e-01  2e-08
 7: -4.5113e+06 -4.4860e+06  8e+04  5e-01  2e-08
 8: -4.4842e+06 -4.4194e+06  9e+04  3e-01  1e-08
 9: -4.4523e+06 -4.3874e+06  8e+04  2e-01  7e-09
10: -4.4268e+06 -4.3674e+06  7e+04  1e-01  4e-09
11: -4.3867e+06 -4.3437e+06  5e+04  6e-02  2e-09
12: -4.3418e+06 -4.3334e+06  3e+04  2e-02  7e-10
13: -4.3301e+06 -4.3304e+06  9e+02  3e-04  1e-11
14: -4.3302e+06 -4.3302e+06  3e+01  7e-06  2e-13
15: -4.3302e+06 -4.3302e+06  3e-01  8e-08  3e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.5449e+06 -4.5449e+06  9e+01  8e+00  3e-07
 1: -4.5449e+06 -4.5449e+06  2e+01  1e+00  4e-0

     pcost       dcost       gap    pres   dres
 0: -4.3641e+06 -4.3641e+06  1e+02  1e+01  2e-07
 1: -4.3641e+06 -4.3641e+06  3e+01  3e+00  6e-08
 2: -4.3641e+06 -4.3641e+06  3e+01  3e+00  6e-08
 3: -4.2226e+06 -4.3765e+06  3e+05  3e+00  6e-08
 4: -4.2559e+06 -4.3587e+06  4e+05  2e+00  4e-08
 5: -4.2723e+06 -4.2797e+06  3e+05  1e+00  3e-08
 6: -4.2211e+06 -4.1690e+06  3e+05  6e-01  2e-08
 7: -4.1529e+06 -4.0516e+06  2e+05  3e-01  8e-09
 8: -4.0876e+06 -3.9972e+06  2e+05  2e-01  5e-09
 9: -4.0169e+06 -3.9633e+06  1e+05  1e-01  2e-09
10: -3.9657e+06 -3.9512e+06  6e+04  4e-02  9e-10
11: -3.9446e+06 -3.9443e+06  9e+03  4e-03  1e-10
12: -3.9427e+06 -3.9427e+06  2e+02  7e-05  2e-12
13: -3.9427e+06 -3.9427e+06  3e+00  8e-07  2e-14
14: -3.9427e+06 -3.9427e+06  3e-02  8e-09  2e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -8.9998e+06 -8.9998e+06  8e+01  8e+00  1e-07
 1: -8.9998e+06 -8.9998e+06  1e+01  1e+00  2e-08
 2: -8.9998e+06 -8.9998e+06  1e+01  9e-01  2e-0

     pcost       dcost       gap    pres   dres
 0: -9.1833e+06 -9.1833e+06  9e+01  1e+01  1e-07
 1: -9.1833e+06 -9.1833e+06  3e+01  3e+00  3e-08
 2: -9.1833e+06 -9.1833e+06  3e+01  3e+00  3e-08
 3: -8.3548e+06 -9.4566e+06  2e+06  3e+00  3e-08
 4: -8.5475e+06 -9.1904e+06  2e+06  2e+00  2e-08
 5: -8.6624e+06 -8.8925e+06  1e+06  1e+00  1e-08
 6: -8.6647e+06 -8.6214e+06  7e+05  5e-01  7e-09
 7: -8.4743e+06 -8.3877e+06  5e+05  2e-01  3e-09
 8: -8.3592e+06 -8.3275e+06  2e+05  7e-02  9e-10
 9: -8.2814e+06 -8.3070e+06  4e+04  3e-03  4e-11
10: -8.2939e+06 -8.2958e+06  2e+03  2e-04  2e-12
11: -8.2952e+06 -8.2953e+06  4e+01  2e-06  3e-14
12: -8.2952e+06 -8.2952e+06  4e-01  3e-08  3e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -8.9612e+06 -8.9612e+06  8e+01  8e+00  1e-07
 1: -8.9612e+06 -8.9612e+06  1e+01  1e+00  2e-08
 2: -8.9612e+06 -8.9612e+06  1e+01  1e+00  2e-08
 3: -8.9577e+06 -8.9502e+06  7e+03  1e+00  2e-08
 4: -8.8786e+06 -8.9199e+06  2e+05  1e+00  2e-0

     pcost       dcost       gap    pres   dres
 0: -3.8383e+06 -3.8383e+06  8e+01  9e+00  3e-07
 1: -3.8383e+06 -3.8383e+06  2e+01  2e+00  7e-08
 2: -3.8383e+06 -3.8383e+06  2e+01  2e+00  6e-08
 3: -3.7420e+06 -3.8149e+06  2e+05  2e+00  6e-08
 4: -3.6923e+06 -3.8218e+06  3e+05  2e+00  6e-08
 5: -3.6971e+06 -3.8141e+06  3e+05  2e+00  5e-08
 6: -3.6792e+06 -3.7756e+06  4e+05  1e+00  4e-08
 7: -3.6877e+06 -3.6968e+06  4e+05  9e-01  3e-08
 8: -3.6602e+06 -3.6041e+06  3e+05  4e-01  2e-08
 9: -3.5845e+06 -3.5215e+06  2e+05  2e-01  8e-09
10: -3.5126e+06 -3.4786e+06  1e+05  8e-02  3e-09
11: -3.4732e+06 -3.4660e+06  3e+04  2e-02  6e-10
12: -3.4615e+06 -3.4619e+06  8e+02  2e-04  7e-12
13: -3.4617e+06 -3.4617e+06  1e+01  3e-06  1e-13
14: -3.4617e+06 -3.4617e+06  1e-01  3e-08  1e-15
Optimal solution found.
准确率为： 217
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [40]:
print("准确率为：",hit_num)
print(real_class_list)
print(predict_class_list)

准确率为： 217
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
[1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 1, 1, 2, 3, 6, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 7, 1, 2, 2, 2, 2, 2, 2, 2, 9, 3, 3, 3, 3, 3, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 9, 3, 3, 4, 4, 9, 

In [30]:
import numpy as np 
list_ = [216,216,218,211,219,218,217,215,216,212]
print(sum(list_)/2500)

var = np.var(list_)
print((var**0.5)/250)

0.8632
0.009765244492586962
